In [4]:

import pandas as pd
import numpy as np


#     Add a column called campaign_id in campaign, where all rows have a value of 1.
#     Create a datetime column called last_contact_date, in the format of "year-month-day", where the year is 2022, and the month and day values are taken from the "month" and "day" columns.
#     Remove any redundant data that might have been used to create new columns, ensuring the columns in each subset of the data match the table displayed in the notebook.
#     Save the three DataFrames to csv files without an index as client.csv, campaign.csv, and economics.csv respectively.
#     Create a variable called client_table, containing SQL code as a string to create a table called client using values from client.csv.
#     Create a variable called campaign_table, containing SQL code as a string to create a table called campaign using values from campaign.csv.
#     Create a variable called economics_table, containing SQL code as a string to create a table called economics using values from economics.csv.
#     In client, campaign, and economic, ensure the final line copies the data from their respective csv files using the following code snippet:
# \copy table_name from 'file_name.csv' DELIMITER ',' CSV HEADER
#     Click the "Submit Project" button to check your answer!

#     Read in bank_marketing.csv as a pandas DataFrame.#  
bank = pd.read_csv('bank_marketing.csv')

#  Rename the column "client_id" to "id" in client (leave as-is in the other subsets)
client = bank[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing', 'loan']]
client.rename(columns={'client_id': 'id'}, inplace=True)
#  Rename "duration" to "contact_duration", "previous" to "previous_campaign_contacts", 
# # "y" to "campaign_outcome", "poutcome" to "previous_outcome", and "campaign" to "number_contacts" in campaign
campaign = bank[['client_id', 'contact', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']]
campaign.rename(columns={'duration': 'contact_duration', 'previous': 'previous_campaign_contacts', 'y': 'campaign_outcome', 'poutcome': 'previous_outcome', 'campaign': 'number_contacts'}, inplace=True)
#  and "euribor3m" to "euribor_three_months" and "nr_employed" to "number_employed" in economics.
economics = bank[['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor3m', 'nr_employed']]
economics.rename(columns={'euribor3m': 'euribor_three_months', 'nr_employed': 'number_employed'}, inplace=True)

#  Clean the "education" column, changing "." to "_" and "unknown" to NumPy's null values.
client['education'] = client['education'].str.replace('.', '_')
client['education'] = client['education'].replace('unknown', np.nan)
#     Remove periods from the "job" column.
client['job'] = client['job'].str.replace('.', '')

#     Convert "success" and "failure" in the "previous_outcome" and "campaign_outcome" columns to binary (1 or 0), along with the changing "nonexistent" to NumPy's null values in "previous_outcome".
campaign['previous_outcome'] = campaign['previous_outcome'].replace({'success': 1, 'failure': 0, 'nonexistent': np.nan})
campaign['campaign_outcome'] = campaign['campaign_outcome'].replace({'success': 1, 'failure': 0})


In [1]:
# Index(['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing', 'loan',

#       'contact', 'month', 'day', 'duration', 'campaign',  'pdays', 'previous', 'poutcome', 
#       
#       'emp_var_rate', 'cons_price_idx',  'cons_conf_idx', 'euribor3m', 'nr_employed', 'y']
